<a href="https://colab.research.google.com/github/drshahizan/Python-big-data/blob/main/assignment/ass6/bdm/Ayam%20Rendang%201/Ayam_Rendang1_Assignment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Python Big Data Assignment 6 - Iowa Liquor Sales Dataset**

Dealing with large dataset using Pandas library with five different techniques mentioned hereby to address the limitations working with big data using Pandas library




**Group Name : Ayam Rendang 1**

**Team Member :**

*   THANEATHARRAN A/L SANTHARASEKARAN (MSC232006)
*   RANJEET A/L THIAGARAJAN (MCS231015)







# **Download the Dataset from directly Kaggle into Google Colab**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/My Drive/kaggle"

In [ ]:
%cd /content/drive/MyDrive/kaggle

/content/drive/MyDrive/kaggle


In [ ]:
!kaggle datasets download -d residentmario/iowa-liquor-sales

100% 730M/731M [00:22<00:00, 40.2MB/s]
100% 731M/731M [00:22<00:00, 34.3MB/s]


In [ ]:
!unzip iowa-liquor-sales.zip && rm iowa-liquor-sales.zip

Archive:  iowa-liquor-sales.zip
  inflating: Iowa_Liquor_Sales.csv   


 # **Import necessary libraries and install required packages**

In [ ]:
import pandas as pd
import numpy as np

# **The total memory used to load the dataset**

In [ ]:
%%time
df = pd.read_csv("/content/drive/MyDrive/kaggle/Iowa_Liquor_Sales.csv")


<timed exec>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1min 6s, sys: 15.4 s, total: 1min 22s
Wall time: 1min 34s


In [ ]:
memory_usage = df.memory_usage().sum()

print("The apparent memory consumption: ",round(memory_usage/1024**3,2), "GigaBytes")

The apparent memory consumption:  2.25 GigaBytes


# **General description of the dataset**

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12591077 entries, 0 to 12591076
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Invoice/Item Number    object 
 1   Date                   object 
 2   Store Number           int64  
 3   Store Name             object 
 4   Address                object 
 5   City                   object 
 6   Zip Code               object 
 7   Store Location         object 
 8   County Number          float64
 9   County                 object 
 10  Category               float64
 11  Category Name          object 
 12  Vendor Number          float64
 13  Vendor Name            object 
 14  Item Number            int64  
 15  Item Description       object 
 16  Pack                   int64  
 17  Bottle Volume (ml)     int64  
 18  State Bottle Cost      object 
 19  State Bottle Retail    object 
 20  Bottles Sold           int64  
 21  Sale (Dollars)         object 
 22  Volume Sold (Lit

# **Strategies for big dataset**

# **Load Less Data**

Load only neccesary data that required based on analysis objectives

Let say we want to study the total sales of liquor then we only need features contribute to answer that specific objective and we can drop others features we are not relevant

In that case only 3 features out of 23 are relevant which are:



*   Bottles Sold           int64
*   Volume Sold (Liters)   float64
*   Volume Sold (Gallons)  float64











In [ ]:
df1 = pd.read_csv("/content/drive/MyDrive/kaggle/Iowa_Liquor_Sales.csv", usecols = ["Bottles Sold","Volume Sold (Liters)","Volume Sold (Gallons)"])

memory_usage = df1.memory_usage().sum()
print("The apparent memory consumption: ",round(memory_usage/1024**3,2), "GigaBytes")

The apparent memory consumption:  0.28 GigaBytes


As you can see the memory consumption decrease from 2.25 GB previously to 0.28 GB

# **Use Chunking**

Let us minimize the memory consumption by divide and conquer method through spliting the dataset into smaller portions which known as chunks and manipulate each chunks one at a time.

In [ ]:
chunk_size = 1000000
chunk_no=1
!mkdir "Chunk_Files"
%cd /content/drive/MyDrive/kaggle/Chunk_Files/
for chunk in pd.read_csv('/content/drive/MyDrive/kaggle/Iowa_Liquor_Sales.csv',chunksize=chunk_size):
    chunk.to_csv('chunk'+str(chunk_no)+'.csv',index=False)
    chunk_no+=1

/content/drive/MyDrive/kaggle/Chunk_Files


In [ ]:
%cd /content/drive/MyDrive/kaggle/Chunk_Files/

/content/drive/MyDrive/kaggle/Chunk_Files


In [ ]:
# Now we have multiple chunk files, and each chunk flie can easily be loaded as a pandas dataframe.

df2 = pd.read_csv('chunk1.csv')

memory_usage = df2.memory_usage().sum()
print("The apparent memory consumption: ",round(memory_usage/1024**3,2), "GigaBytes")
df2.info()

The apparent memory consumption:  0.18 GigaBytes
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Invoice/Item Number    1000000 non-null  object 
 1   Date                   1000000 non-null  object 
 2   Store Number           1000000 non-null  int64  
 3   Store Name             1000000 non-null  object 
 4   Address                1000000 non-null  object 
 5   City                   1000000 non-null  object 
 6   Zip Code               1000000 non-null  object 
 7   Store Location         1000000 non-null  object 
 8   County Number          998952 non-null   float64
 9   County                 998952 non-null   object 
 10  Category               999711 non-null   float64
 11  Category Name          999415 non-null   object 
 12  Vendor Number          1000000 non-null  int64  
 13  Vendor Name            1

As a result of chunking method, it has reduced the memory usage from 2.25 GB to 0.18 GB by subdividing dataset into smaller parts.

# **Optimize Data Types**

In [ ]:
# check for the dtypes of the columns that are int64

df.select_dtypes(include=np.number).dtypes
df.select_dtypes(include="int64").dtypes
numeric_columns = df.select_dtypes(include="int64").columns
print(numeric_columns)


Index(['Store Number', 'Item Number', 'Pack', 'Bottle Volume (ml)',
       'Bottles Sold'],
      dtype='object')


In [ ]:
# Find the minimum and maximum values for each columns

min_values = df[numeric_columns].min()
max_values = df[numeric_columns].max()

print("Minimum values:")
print(min_values)

print("\nMaximum values:")
print(max_values)


Minimum values:
Store Number          2106
Item Number            101
Pack                     1
Bottle Volume (ml)       0
Bottles Sold             0
dtype: int64

Maximum values:
Store Number            9932
Item Number           999275
Pack                     600
Bottle Volume (ml)    378000
Bottles Sold           15000
dtype: int64


In [ ]:
# Convert Integer data type into smaller data type (uint8, uint16)
# Define columns and their corresponding dtypes based on the min and max values

columns_and_dtypes = {}
for cols in numeric_columns:
  temp = {cols: 'int8' if 0 <= df[cols].min() and df[cols].max() <= 255 else 'int16'}
  columns_and_dtypes.update(temp)


# Apply the conversion to each column
df3 = df.astype(columns_and_dtypes)


In [ ]:
# Check the DF Types after optimized Data Types
df3.dtypes

Invoice/Item Number       object
Date                      object
Store Number               int16
Store Name                object
Address                   object
City                      object
Zip Code                  object
Store Location            object
County Number            float64
County                    object
Category                 float64
Category Name             object
Vendor Number            float64
Vendor Name               object
Item Number                int16
Item Description          object
Pack                       int16
Bottle Volume (ml)         int16
State Bottle Cost         object
State Bottle Retail       object
Bottles Sold               int16
Sale (Dollars)            object
Volume Sold (Liters)     float64
Volume Sold (Gallons)    float64
dtype: object

In [ ]:
memory_usage = df3.memory_usage().sum()
print("The apparent memory consumption: ",round(memory_usage/1024**3,2), "GigaBytes")

The apparent memory consumption:  1.9 GigaBytes


Now you can see that the memory utilization to load the dataset reduced from 2.25 GB intially to just 1.9 GB.

# **Sampling**

In [ ]:
# Random sampling of 10% of Dataset
sample1 = df.sample(frac=0.10)
sample1

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
1635544,S06443000012,07/06/2012,4295,Nat Food Mart #1 / Hubbell,3804 HUBBELL,DES MOINES,50317,"3804 HUBBELL\nDES MOINES 50317\n(41.630651, -9...",77.0,Polk,...,40629,UV Whipped Mini,6,500,$4.88,$7.32,1,$7.32,0.50,0.13
10806151,INV-02781500007,01/18/2017,2548,Hy-Vee Food Store / Altoona,100 8th Street SW,Altoona,50009.0,"100 8th Street SW\nAltoona 50009\n(41.644041, ...",77.0,POLK,...,65256,Jagermeister Liqueur,12,750,$12.05,$18.08,1,$216.96,0.75,0.19
12019906,INV-06435500022,08/02/2017,2618,Hy-Vee Drugstore #6 / Cedar Rapids,505 Boyson Rd NE,Cedar Rapids,52402.0,505 Boyson Rd NE\nCedar Rapids 52402\n(42.0467...,57.0,LINN,...,34001,Absolut Swedish Vodka 80 Prf Mini,10,600,$7.92,$11.88,1,$11.88,0.60,0.15
8559412,S30071100030,01/07/2016,3670,Wal-Mart 2827 / Coralville,2801 COMMERCE DR,CORALVILLE,52241,2801 COMMERCE DR\nCORALVILLE 52241\n(41.695806...,52.0,Johnson,...,69637,Dr. Mcgillicuddy's Cherry Schnapps,12,1000,$11.00,$16.50,12,$198.00,12.00,3.17
1950648,S13816900011,08/08/2013,2552,Hy-Vee Food Store #3 / Cedar Rapids,20 WILSON AVENUE WEST,CEDAR RAPIDS,52404,20 WILSON AVENUE WEST\nCEDAR RAPIDS 52404\n(41...,57.0,Linn,...,41681,Uv Red (cherry) Vodka,6,1750,$8.48,$13.98,6,$83.88,10.50,2.77
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9095230,S31967900047,04/25/2016,2584,Hy-Vee Food Store / Fort Madison,2606 AVENUE L,FORT MADISON,52627,"2606 AVENUE L\nFORT MADISON 52627\n(40.626988,...",56.0,Lee,...,67006,Midori Melon Liqueur,12,750,$11.50,$17.25,2,$34.50,1.50,0.40
3171341,S21946300071,10/22/2014,2548,Hy-Vee Food Store / Altoona,100 8TH STREET SW,ALTOONA,50009,"100 8TH STREET SW\nALTOONA 50009\n(41.644041, ...",77.0,Polk,...,57148,Chi-Chi's Margarita W/tequila,6,1750,$6.00,$9.25,6,$55.50,10.50,2.77
7492937,S22832200018,12/09/2014,2353,Cresco Liquor Store,708 2ND AVE SE,CRESCO,52136,"708 2ND AVE SE\nCRESCO 52136\n(43.371131, -92....",45.0,Howard,...,11777,Black Velvet,12,1000,$6.63,$9.95,24,$238.80,24.00,6.34
6218696,S06056000014,06/13/2012,2963,Dahl's / Clive,15500 HICKMAN,CLIVE,50325,"15500 HICKMAN\nCLIVE 50325\n(41.614714, -93.83...",77.0,Polk,...,12408,Canadian Ltd Whisky,6,1750,$7.79,$12.29,6,$73.74,10.50,2.77


In [ ]:
memory_usage = sample1.memory_usage().sum()
print("The apparent memory consumption: ",round(memory_usage/1024**3,2), "GigaBytes")


The apparent memory consumption:  0.23 GigaBytes


In [ ]:
sample2 = df.sample(n=1000000,axis=0)
sample2

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,...,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
7896591,S26871500002,07/21/2015,4448,Kum & Go #572 / URBANDALE,4860 NW URBANDALE DR,URBANDALE,50322,4860 NW URBANDALE DR\nURBANDALE 50322\n,77.0,Polk,...,36308,Hawkeye Vodka,6,1750,$7.17,$10.76,6,$64.56,10.50,2.77
294676,S05377800013,05/07/2012,4577,Casey's General Store #2551 / Woodwa,"906, S MAIN ST",WOODWARD,50276,"906, S MAIN ST\nWOODWARD 50276\n(41.849042, -9...",25.0,Dallas,...,5345,Johnnie Walker Red,24,375,$7.50,$11.24,6,$67.44,2.25,0.59
3988716,S23803700035,02/02/2015,4622,Quik Trip #544 / SE 14th DM,"3941, SE 14th ST",DES MOINES,50320,"3941, SE 14th ST\nDES MOINES 50320\n(41.548497...",77.0,Polk,...,35776,Barton Vodka Mini,10,600,$3.92,$5.88,2,$11.76,1.20,0.32
8985246,S31652500070,04/06/2016,4509,A J'S LIQUOR II,2515 CHAMBERLAIN,AMES,50010,"2515 CHAMBERLAIN\nAMES 50010\n(42.021461, -93....",85.0,Story,...,40313,New Amsterdam Peach,6,1750,$13.32,$19.98,42,$839.16,73.50,19.42
9310443,S32573400001,05/31/2016,3621,"Jensen Liquors, Ltd.",615 2nd Ave,Sheldon,51201,"615 2nd Ave\nSheldon 51201\n(43.184614, -95.85...",71.0,O'Brien,...,902165,Phillips Raspberry Long Island Ice Tea,6,1750,$8.30,$12.45,12,$149.40,21.00,5.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8832224,S31189500057,03/14/2016,3385,Sam's Club 8162 / Cedar Rapids,2605 BLAIRS FERRY RD NE,CEDAR RAPIDS,52402,2605 BLAIRS FERRY RD NE\nCEDAR RAPIDS 52402\n(...,57.0,Linn,...,43338,Captain Morgan Spiced Rum,6,1750,$18.00,$27.00,72,$1944.00,126.00,33.29
11277845,INV-04256200035,04/10/2017,2847,CVS Pharmacy #10032 / Marion,"3499, 7th Ave",Marion,52302.0,"3499, 7th Ave\nMarion 52302\n(42.035661, -91.5...",57.0,LINN,...,64868,Fireball Cinnamon Whiskey,6,1750,$15.33,$23.00,1,$23.00,1.75,0.46
5856721,S21550900032,10/01/2014,2512,Hy-Vee Wine and Spirits / Iowa City,1720 WATERFRONT DR,IOWA CITY,52240,1720 WATERFRONT DR\nIOWA CITY 52240\n(41.64277...,52.0,Johnson,...,36034,Pearl Peach,12,750,$7.86,$11.79,12,$141.48,9.00,2.38
14043,S24325100018,03/04/2015,4767,Casey's General Store #3075 / Ankeny,"6981, NE 14TH ST",ANKENY,50023,"6981, NE 14TH ST\nANKENY 50023\n(41.687898, -9...",77.0,Polk,...,36903,Mccormick Vodka,48,200,$1.13,$1.70,4,$6.80,0.80,0.21


In [ ]:
memory_usage = sample2.memory_usage().sum()
print("The apparent memory consumption: ",round(memory_usage/1024**3,2), "GigaBytes")

The apparent memory consumption:  0.19 GigaBytes


After we apply the two different sampling techniques, we can observe that the memory consumption reduced from 2.25 GB+ to 0.23 GB and 0.19 GB for the respective technique.  

# **Parallelize with Dask**

In [ ]:
%%time
import dask.dataframe as dd
ddf = dd.read_csv("/content/drive/MyDrive/kaggle/Iowa_Liquor_Sales.csv")
ddf

CPU times: user 12.6 ms, sys: 844 µs, total: 13.4 ms
Wall time: 15.9 ms


,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
npartitions=54,,,,,,,,,,,,,,,,,,,,,,,,
,object,object,int64,object,object,object,object,object,int64,object,float64,object,int64,object,int64,object,int64,int64,object,object,int64,object,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [ ]:
ddf.columns

Index(['Invoice/Item Number', 'Date', 'Store Number', 'Store Name', 'Address',
       'City', 'Zip Code', 'Store Location', 'County Number', 'County',
       'Category', 'Category Name', 'Vendor Number', 'Vendor Name',
       'Item Number', 'Item Description', 'Pack', 'Bottle Volume (ml)',
       'State Bottle Cost', 'State Bottle Retail', 'Bottles Sold',
       'Sale (Dollars)', 'Volume Sold (Liters)', 'Volume Sold (Gallons)'],
      dtype='object')

The loading time took for the dataset using Pandas is in minutes while it's only took in miliseconds in Dask which is a lot more faster than using Pandas

# **Comparative Analysis**

| No. | Usual Pandas Method | Advanced Strategies | Result |
   | --- | ------------------ | -------------------- | ------ |
   | 1.  | Memory usage 2.25GB | **Load Less Data**: Memory usage 0.28GB | Memory usage reduced by 87.56% |
   | 2.  | Memory usage 2.25GB | **Use Chunking**: Memory usage 0.18GB | Memory usage reduced by 92.0% |
   | 3.  | Data Frame Size 2.25GB | **Optimize Data Types**: Data Frame Size 1.9 GB | Data frame size reduced by 15.56% |
   | 4.  | Memory usage 2.25GB | **Sampling**: 10% sampling; Memory usage 0.23 GB <br> 10000 rows sampling; Memory usage 0.19 GB | Memory usage reduced by 89.78% <br> Memory usage reduced by 91.56% |
   | 5.  | Reading Data frame: 1min 34s  | **Parallelize with Dask**: Reading Data frame: 15.9 ms  | Processing time reduced by 99.9%  |

Through the comparative analysis it clearly shows that the usual pandas method has its own limitation when handling big datasets. It has limitation in RAM, memory usage and processinf time. The advance strategies helps to reduce the memory usage, fasten the processing time, reduce the RAM percentage and other ways to analyse big datasets.

# **Conclusion**

Ultimately, advanced methods enable us to use Pandas for handling huge amounts of data in the following ways: -

**1) Load less data**: Optimising memory consumption and speeding processes can be achieved by loading only the certain columns or rows needed for the analysis. But because it doesn't represent the whole dataset, it can have an impact on the choice.

**2) Chunking**: In order to prevent memory problems, chunks enable data to be processed in smaller portions. This is especially helpful for procedures that don't need the whole dataset to be open in memory at once. Still, working with chunks may make some processes more complicated.

**3) Optimized data types**: Selecting the most memory-efficient data types for every column will significantly decrease memory consumption and simplify computations by reducing the amount of data needed. As a result, it may increase overall effectiveness.

**4) Sampling**: Sampling can provide insight into and aid in comprehending the dataset's general properties. However, sampling may not fully represent the entire population, meaning that some rare occurrences or changes may go unnoticed.

**5) Parallelize with Dask**: Dask is a parallel computing platform that allows compute scaling and parallelization.  Dask's ability to parallelize calculations is one of its primary characteristics; it makes it possible to work with datasets that are larger than memory by dividing them into smaller jobs that can be completed in parallel.

In conclusion, we can consider optimizing the data types or continuing over the data to process all of our data. Dask can parallelize Pandas if our dataset is still too big and requires too much processing time, as its processing time has been reduced by 99.9%. Together, these techniques seek to improve processing speed, use less memory, and offer effective methods for managing big information without taxing the system's capacity.